Este script genera graficos de frecuencia instantanea de la DOE vs. tiempo durante el estimulo. Superpone cada estimulacion y las separa en sub-plots por hora consecutiva de estimulacion. Tiene la opcion de separar entre el trial de la mañana y la tarde.

Toma los archivos .pkl obtenidos de 'EOD_analysis.py' y los .csv registrados con bonsai con los timestamps de los on y off del objeto. Puede ser adaptado para graficar raster-plot, graficar todas las horas de estimulacion en uno y cambiar los rangos de tiempo de interes para plotear. 

In [203]:
#importamos los paquetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import pickle
from datetime import datetime, timedelta
from scipy.stats import zscore

#cambiamos la carpeta de trabajo y cargamos los archivos
data_folder = r'D:\Datos G. omarorum\Fish6\Objeto\Trial 1 y 2\raw' #cambiar a ruta con archivos .bin
os.chdir(data_folder)

files_vid = sorted(glob.glob('*.h5'))
files_EOD = sorted(glob.glob('*.bin'))
obj_coordinates = [291, 213]
#definimos parametros
sf = 10000


In [204]:
#cargamos el archivo de FB-DOE
with open('fish6_FB-DOE.pkl', 'rb') as file:   #cambiar al nombre apropiado de archivo
        FB_doe = pickle.load(file)

A continuacion tenemos algunas celdas que pre-procesan nuestros datos y filtran los archivos para solo utilizar los que contienen registros durante la estimulacion.

In [205]:
# generamos la lista files_start que contiene las timestamps en formato datetime del comienzo de cada uno de los archivos
files_start = [datetime.strptime(key[:-1], '%Y-%m-%dT%H_%M_%S') for key in FB_doe['FB-DOE'].keys()]

print(files_start)

[datetime.datetime(2024, 2, 2, 17, 1, 11), datetime.datetime(2024, 2, 2, 17, 21, 12), datetime.datetime(2024, 2, 2, 17, 41, 12), datetime.datetime(2024, 2, 2, 18, 1, 12), datetime.datetime(2024, 2, 2, 18, 21, 12), datetime.datetime(2024, 2, 2, 18, 41, 12), datetime.datetime(2024, 2, 2, 19, 1, 12), datetime.datetime(2024, 2, 2, 19, 21, 12), datetime.datetime(2024, 2, 2, 19, 41, 13), datetime.datetime(2024, 2, 2, 20, 1, 13), datetime.datetime(2024, 2, 2, 20, 21, 13), datetime.datetime(2024, 2, 2, 20, 41, 13), datetime.datetime(2024, 2, 2, 21, 1, 13), datetime.datetime(2024, 2, 2, 21, 21, 13), datetime.datetime(2024, 2, 2, 21, 41, 13), datetime.datetime(2024, 2, 2, 22, 1, 14), datetime.datetime(2024, 2, 2, 22, 21, 14), datetime.datetime(2024, 2, 2, 22, 41, 14), datetime.datetime(2024, 2, 2, 23, 1, 14), datetime.datetime(2024, 2, 2, 23, 21, 14), datetime.datetime(2024, 2, 2, 23, 41, 14), datetime.datetime(2024, 2, 3, 0, 1, 15), datetime.datetime(2024, 2, 3, 0, 21, 15), datetime.datetime(20

In [206]:
from datetime import datetime, timedelta

def generate_timestamps(start_date, start_time, hours):
    # Define the start and end times
    start_time = datetime(start_date.year, start_date.month, start_date.day, start_time, 0, 0)
    end_time = start_time + timedelta(hours=hours)  # 8 hours from 9 PM to 5 AM

    # Initialize an empty list to store timestamps
    timestamps = []

    # Generate timestamps at one-minute intervals
    current_time = start_time
    while current_time <= end_time:
        timestamps.append(current_time)
        current_time += timedelta(minutes=1)

    return timestamps

# Generamos las timestamps:
given_date = files_start[48] 
timestamps = generate_timestamps(given_date, start_time=9, hours=2)


In [207]:
import random
from datetime import datetime, timedelta

# Function to randomly sample 60 timestamps
def sample_random_timestamps(timestamps, seed):
    # Ensure that we have at least 60 timestamps available
    if len(timestamps) < 60:
        raise ValueError("Not enough timestamps to sample from.")

    random.seed(seed)
    # Sample 60 timestamps randomly
    sampled_timestamps = random.sample(timestamps, 60)
    return sampled_timestamps

# Sample 60 random timestamps
random_timestamps = sorted(sample_random_timestamps(timestamps, seed=13))

In [208]:
print(len(files_start), len(FB_doe['FB-DOE'].keys()))

55 55


In [209]:
nueve = datetime(given_date.year, given_date.month, given_date.day, 9, 0, 0)
once = nueve + timedelta(hours=2) 

keys = [x for i, x in enumerate(FB_doe['FB-DOE'].keys()) if files_start[i] > nueve and files_start[i] < once]
k_idx = [i for i, x in enumerate(FB_doe['FB-DOE'].keys()) if files_start[i] > nueve and files_start[i] < once]
FB_DOE = {key: FB_doe['FB-DOE'][key] for key in keys}
Peak_time = {key: FB_doe['Peak-time'][key] for key in keys}# guardamos los peak-times en otra variable
n_files = len(keys)

In [210]:
files_start = [files_start[x] for x in k_idx] #nos quedamos solo con los timestamps de los archivos de interes
files_start


[datetime.datetime(2024, 2, 3, 9, 1, 19),
 datetime.datetime(2024, 2, 3, 9, 21, 19),
 datetime.datetime(2024, 2, 3, 9, 41, 19),
 datetime.datetime(2024, 2, 3, 10, 1, 19),
 datetime.datetime(2024, 2, 3, 10, 21, 19),
 datetime.datetime(2024, 2, 3, 10, 41, 19)]

In [211]:
#organizamos cada on segun su archivo de registro
files = pd.DataFrame(np.zeros(shape=(20, n_files)), columns=FB_DOE.keys())  # Usamos Int64Dtype para que pueda haber nans en columnas de int

for i, column in enumerate(files.columns[:-1], start=0):
    start = files_start[i]
    end = files_start[i+1]
    s = 0
    for j in range(len(random_timestamps)):
        condition = (start - random_timestamps[j]).total_seconds() < 0 and (end - random_timestamps[j]).total_seconds() > 0
        if condition:
            files.loc[s, column] = j
            s += 1

files = files.dropna(how='all')
files.replace(0, np.nan, inplace=True) # como inicializamos con una matriz de 0s, si hay algun archivo con menos de 5 ons vamos a tener 0s donde no deben haber, entonces los convertimos a nan
files.iloc[0, 0] = 0 # el primer objeto tiene que ser un 0
files = files.dropna(how='all')

files

,2024-02-03T09_01_19.,2024-02-03T09_21_19.,2024-02-03T09_41_19.,2024-02-03T10_01_19.,2024-02-03T10_21_19.,2024-02-03T10_41_19.
0,0.0,8.0,19.0,26.0,35.0,NaN
1,2.0,9.0,20.0,27.0,36.0,NaN
2,3.0,10.0,21.0,28.0,37.0,NaN
3,4.0,11.0,22.0,29.0,38.0,NaN
4,5.0,12.0,23.0,30.0,39.0,NaN
5,6.0,13.0,24.0,31.0,40.0,NaN
6,7.0,14.0,25.0,32.0,41.0,NaN
7,NaN,15.0,NaN,33.0,42.0,NaN
8,NaN,16.0,NaN,34.0,43.0,NaN
9,NaN,17.0,NaN,NaN,44.0,NaN


In [212]:
#inicializamos las listas
EOD_peaks_on = []
time_EOD_all  = []
time_obj_all = []
EOD_f_on = []

In [213]:
for k, key in enumerate(files.keys()): #loopeamos entre los archivos de interes
    midnight = files_start[k].replace(hour=0, minute=0, second=0, microsecond=0) #definimos la media noche para el dia donde se registro ese archivo
    start = abs(midnight - files_start[k]).total_seconds() # calculamos el tiempo de inicio del archivo en segundos totales respecto de las 00 para poder compararla
    EOD = np.fromfile(files_EOD[k],dtype=np.int16)
    time_EOD = np.linspace(start=start, stop=start+len(EOD)/sf, num=len(EOD))
    del EOD

    EOD_peaks = np.array(Peak_time[key])
    EOD_freq = np.array(FB_DOE[key])
    time_obj = np.zeros((20)) #inicializamos nuestra matriz de tiempo de prendida de obj (cada archivo puede tener maximo 5 ons, por eso las dimensiones)

    l=0
    for i in files.iloc[:,k]:
        if not np.isnan(i):
                s = abs(midnight - random_timestamps[int(i)]).total_seconds() #calculamos el inicio del on
                time_obj[l] = s
                time_obj_all.append(s) #guardamos el tiempo de este on en nuestra lista de tiempos de objeto
                time_EOD_all.append(time_EOD) #guardamos una copia de time_EOD para cada on 
                l+=1
        
    time_peaks = time_EOD[EOD_peaks]
    EOD_zscore = zscore(EOD_freq)
    for j in range(time_obj.shape[0]):
        if not time_obj[j]==0:
            range_on = [time_obj[j]-.5, time_obj[j]+10] #definimos el rango de interes: 1/2 segundo antes que sea el on y 2 segundos despues
            #con = [range_on[0] <= t and t <= range_on[1] for t in videoTime]
            condition = [range_on[0] <= time and time <= range_on[1] for time in time_peaks]

            EOD_peaks_on.append(EOD_peaks[condition])
            EOD_f_on.append(EOD_zscore[condition[:-1]])


    print('termino archivo ' + str(k))

termino archivo 0
termino archivo 1
termino archivo 2
termino archivo 3
termino archivo 4
termino archivo 5


In [214]:
len(EOD_zscore)

46365

In [215]:

EOD_zscore = EOD_f_on
peaks = EOD_peaks_on
n_per_on = []
if len(EOD_zscore)>0 :
    for z_score, peak in zip(EOD_zscore, peaks):
        peak = peak[:len(z_score)]
        if len(z_score)>0:
            events =[]
            for k in np.arange(1,len(z_score)-3):
                is_event = z_score[k] > 1.5 and z_score[k+1] > 1.5 and z_score[k+2] > 1.5 and z_score[k-1] < 1.5
                if is_event: 
                    events.append(k) 
            i_novel = peak[[k for k in events]]
            novel = z_score[events]
            n_per_on.append(len(i_novel)/len(peak))
            
count = 0
for i in n_per_on:
    if i>0:
        count+=1 

print((count/len(n_per_on))*100)

20.454545454545457


In [58]:
len(n_per_on)

49